In [1]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=9e8c9fe307988a584dbbd9e421e7cd03832ce8479efc87e9af35a2f32a392b9a
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [2]:
from rouge_score import rouge_scorer
import json

import pandas as pd
pd.set_option("display.max_colwidth", None)
from itertools import product
from statistics import mean

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
testset_path = '/content/drive/MyDrive/BioNLP-2023/bionlp-workshop-2023-shared-task-1a-problem-list-summarization-1.0.0/'

In [5]:
# Pagerank predictions
ground_truth = pd.read_csv(testset_path + "BioNLP2023-1A-newTest.csv")

In [ ]:
jsonl_file = 'system.jsonl'
txt_file = 'system.txt'
p = inflect.engine()

with open(jsonl_file, 'r') as f_in, open(txt_file, 'w') as f_out:
    for line in f_in:
        data = json.loads(line)
        words = data.split()
        singular_words = []
        for word in words:
            singular_word = p.singular_noun(word) or word
            singular_words.append(singular_word)
        singular_data = ' '.join(singular_words)
        # Remove all the "#" characters
        #singular_data = singular_data.replace("#", "")
        f_out.write(singular_data + '\n')

In [18]:
import json
import inflect
import pandas as pd
import numpy as np

def add_txt_column(df, txt_file, column_name):
    """
    Adds a text file as a new column to a DataFrame.

    Parameters:
        df (pandas.DataFrame): DataFrame to add the column to.
        txt_file (str): Path to the text file.
        column_name (str): Name of the new column.

    Returns:
        pandas.DataFrame: DataFrame with the new column.
    """
    with open(txt_file, 'r') as f:
        data = f.read().splitlines()

    df[column_name] = data

    return df

def pairwise_rouge(df, column_name_1, column_name_2):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores = []
    for i in range(len(df)):
        score = scorer.score(df.iloc[i][column_name_1], df.iloc[i][column_name_2])
        
        scores.append({
            "document": i,
            "rouge_1": score["rouge1"].fmeasure,
            "rouge_2": score["rouge2"].fmeasure,
            "rouge_L": score["rougeL"].fmeasure
        })
        
    return scores

In [ ]:
# add a JSONL file as a column to the DataFrame
df = add_txt_column(ground_truth, "system.txt", "generated_summary")

# compare the contents of the "summary" column pairwise using Rouge
scores = pairwise_rouge(df, "summary", "generated_summary")

# print the pairwise Rouge scores
for score in scores:
    print(score)

In [ ]:
df.to_csv(testset_path + "BioNLP2023-1A-newTest_compare.csv", index=False)

In [6]:
df =pd.read_csv(testset_path + "BioNLP2023-1A-newTest_compare.csv")

In [14]:
# compare the contents of the "summary" column pairwise using Rouge
scores = pairwise_rouge(df, "Summary", "generated_summary")

In [19]:
# calculate the average Rouge scores
rouge_1_scores = [score["rouge_1"] for score in scores]
rouge_2_scores = [score["rouge_2"] for score in scores]
rouge_l_scores = [score["rouge_L"] for score in scores]

avg_rouge_1 = np.mean(rouge_1_scores)
avg_rouge_2 = np.mean(rouge_2_scores)
avg_rouge_l = np.mean(rouge_l_scores)

print("Average Rouge-1 score:", avg_rouge_1)
print("Average Rouge-2 score:", avg_rouge_2)
print("Average Rouge-L score:", avg_rouge_l)

Average Rouge-1 score: 0.27262912804913153
Average Rouge-2 score: 0.09779431087128522
Average Rouge-L score: 0.23630261669371613
